## Homework 5.4 

a)
combine the title and abstract. 

bigrams are sequences of 2 words. 
unigrams are single words.
Therefore, the tokens are twice as long for text that is tokenized into bigrams

In [ ]:
# start of code:
